<a href="https://colab.research.google.com/github/sccmst/NLUModelOnColab/blob/GPT2-ContentExtension/GPT-ContentExtension/Fine_tune_GPT_2_Model_with_Huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
with open("/root/.kaggle/kaggle.json", "w") as f:
  f.write('{"username":"cstsmc","key":"41599cdf1be98f2d65480249a887043b"}')

!chmod 600 /root/.kaggle/kaggle.json

## **What are we going to do:**

- load the dataset from kaggle
- prepare the dataset and build a ``TextDataset``
- load the pre-trained GPT-2 model and tokenizer
- initialize ``Trainer`` with ``TrainingArguments``
- train and save the model
- test the model

In [ ]:
!pip install transformers>=4.2.2

In [ ]:
from transformers import set_seed
# Set seed for reproducibility.
set_seed(123)


In [ ]:
!nvidia-smi

Wed Dec 14 06:24:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    30W /  70W |  14790MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Prepare the dataset and build a ``TextDataset``

The next step is to extract the instructions from all recipes and build a `TextDataset`. The `TextDataset` is a custom implementation of the [Pytroch `Dataset` class](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class) implemented by the transformers library. If you want to know more about Dataset in Pytroch you can check out this [youtube video](https://www.youtube.com/watch?v=PXOzkkB5eH0&ab_channel=PythonEngineer).

First, we are going to split the `recipes.json` into a `train` and `test` section and extract `Instructions` from the recipes and write them into a `train_dataset.txt` and `test_dataset.txt`

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
#api.dataset_download_files("sterby/german-recipes-dataset",path="./", unzip=True)
api.dataset_download_files("terrychanorg/chinese-simplified-xlsum-v2", path="./", unzip=True)

In [ ]:
import re
import json
from sklearn.model_selection import train_test_split


with open('./chinese_traditional_XLSum_v2.0/chinese_traditional_val.jsonl') as f:
    # row = json.load(f)
    data = []
    for line in f.readlines():
      line.replace("\n","")
      data.append(json.loads(line))

def build_text_files(data_json, dest_path):
    with open(dest_path, 'w') as f:
      data = []
      for texts in data_json:
          title = str(texts['title']).strip()
          text = str(texts['text']).strip()
          summary = str(texts['summary']).strip()
          data.append(f"{summary}BEG;END{text}")
          data.append(f"{title}BEG;END{text}")
      f.write("\n".join(data))



In [ ]:
train, test = train_test_split(data,test_size=0.15)
build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')
print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))


Train dataset length: 3969
Test dataset length: 701


the next step is to download the tokenizer, which we use. We use the tokenizer from the `german-gpt2` model on [huggingface](https://huggingface.co/anonymous-german-nlp/german-gpt2).

In [ ]:
MODEL_NAME = "uer/gpt2-chinese-cluecorpussmall"
# MODEL_NAME = "ckiplab/gpt2-base-chinese"
# MODEL_NAME = "gpt2"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Downloading:   0%|          | 0.00/217 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/577 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--uer--gpt2-chinese-cluecorpussmall/snapshots/7c87595de655dc7b0fbfaa545ae413a118063d0b/config.json
Model config GPT2Config {
  "_name_or_path": "uer/gpt2-chinese-cluecorpussmall",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "output_past": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--uer--gpt2-chinese-cluecorpussmall/snapshots/7c87595de655dc7b0fbfaa545ae413a118063d0b/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--uer--gpt2-chinese-cluecorpussmall/snapshots/7c87595de655dc7b0fbfaa545ae413a118063d0b/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--uer--gpt2-chinese-cluecorpussmall/snapshots/7c87595de655dc7b0fbfaa545ae413a118063d0b/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--uer--gpt2-chinese-cluecorpussmall/snapshots/7c87595de655dc7b0fbfaa545ae413a118063d0b/config.json
Model config GPT2Config {
  "_name_or_path": "uer/gpt2-chinese-cluecorpussmall",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadMod

In [ ]:
from torch.utils.data import Dataset
from typing import Any, Optional
import torch
import os
import logging
from filelock import FileLock
import time
import pickle
logger = logging.getLogger()
class TextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach soon.
    """

    def __init__(
        self,
        tokenizer: Any,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        cache_dir: Optional[str] = None,
    ):

        if os.path.isfile(file_path) is False:
            raise ValueError(f"Input file path {file_path} not found")

        block_size = block_size - tokenizer.num_special_tokens_to_add(pair=False)

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            cache_dir if cache_dir is not None else directory,
            f"cached_lm_{tokenizer.__class__.__name__}_{block_size}_{filename}",
        )

        # Make sure only the first process in distributed training processes the dataset,
        # and the others will use the cache.
        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):

                logger.info(f"Creating features from dataset file at {directory}")

                self.examples = []
                with open(file_path, encoding="utf-8") as f:
                    # text = f.read()
                    texts = f.readlines()

                # tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))
                # for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
                    # self.examples.append(tokenizer.build_inputs_with_special_tokens(tokenized_text[i : i + block_size])
                # )

                for text in texts:
                  tokenedtext = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))
                  for i in range(0, len(tokenedtext) - block_size + 1, block_size):
                    self.examples.append(tokenizer.build_inputs_with_special_tokens(tokenedtext[i : i + block_size]))
                
                
                # Note that we are losing the last truncated example here for the sake of simplicity (no padding)
                # If your dataset is small, first you should look for a bigger one :-) and second you
                # can change this behavior by adding (model specific) padding.

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    f"Saving features into cached file {cached_features_file} [took {time.time() - start:.3f} s]"
                )

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i], dtype=torch.long)
        # return self.examples[i]


In [ ]:
from transformers import DataCollatorForLanguageModeling
# from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
# https://github.com/huggingface/transformers/blob/main/src/transformers/data/datasets/language_modeling.py
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=512)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=512)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (2764 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
train_dataset[:3]

tensor([[ 101, 5401, 1751,  ..., 4294, 1762,  102],
        [ 101, 5401, 1751,  ..., 4525, 3280,  102],
        [ 101, 2111, 2094,  ...,  889, 6303,  102]])

# login to huggingface

In [ ]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


# Initialize `Trainer` with `TrainingArguments` and GPT-2 model

The [Trainer](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Trainer) class provides an API for feature-complete training. It is used in most of the [example scripts](https://huggingface.co/transformers/examples.html) from Huggingface. Before we can instantiate our `Trainer` we need to download our GPT-2 model and create a [TrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) to access all the points of customization during training. In the `TrainingArguments`, we can define the Hyperparameters we are going to use in the training process like our `learning_rate`, `num_train_epochs`, or  `per_device_train_batch_size`. A complete list can you find [here](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments).

In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained(MODEL_NAME)


training_args = TrainingArguments(
    output_dir="./gpt2-reporter", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=2, # number of training epochs
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=8,  # batch size for evaluation
    evaluation_strategy="steps",
    eval_steps = 400, # Number of update steps between two evaluations.
    logging_steps= 400,
    save_steps = 800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    load_best_model_at_end=True,
    save_total_limit=3,
    learning_rate=5e-5,
    push_to_hub=True,
    hub_model_id="theta/gpt2-reporter"
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--uer--gpt2-chinese-cluecorpussmall/snapshots/7c87595de655dc7b0fbfaa545ae413a118063d0b/config.json
Model config GPT2Config {
  "_name_or_path": "uer/gpt2-chinese-cluecorpussmall",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner":

Downloading:   0%|          | 0.00/421M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--uer--gpt2-chinese-cluecorpussmall/snapshots/7c87595de655dc7b0fbfaa545ae413a118063d0b/pytorch_model.bin
All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at uer/gpt2-chinese-cluecorpussmall.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/content/./gpt2-reporter is already a clone of https://huggingface.co/theta/gpt2-reporter. Make sure you pull the latest changes with `repo.git_pull()`.


# Train and save the model

To train the model we can simply run `Trainer.train()`.

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11395
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2850
  Number of trainable parameters = 102068736


Step,Training Loss,Validation Loss
400,2.769400,2.575113
800,2.633600,2.531773
1200,2.556400,2.507054
1600,2.482000,2.499257
2000,2.424300,2.491004
2400,2.400900,2.485012
2800,2.386500,2.481910


***** Running Evaluation *****
  Num examples = 2038
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2038
  Batch size = 8
Saving model checkpoint to ./gpt2-reporter/checkpoint-800
Configuration saved in ./gpt2-reporter/checkpoint-800/config.json
Model weights saved in ./gpt2-reporter/checkpoint-800/pytorch_model.bin
Several commits (2) will be pushed upstream.
***** Running Evaluation *****
  Num examples = 2038
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2038
  Batch size = 8
Saving model checkpoint to ./gpt2-reporter/checkpoint-1600
Configuration saved in ./gpt2-reporter/checkpoint-1600/config.json
Model weights saved in ./gpt2-reporter/checkpoint-1600/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2038
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2038
  Batch size = 8
Saving model checkpoint to ./gpt2-reporter/checkpoint-2400
Configuration saved in ./gpt2-reporter/checkpoint-2400/config.json
Model weights 

TrainOutput(global_step=2850, training_loss=2.519820521839878, metrics={'train_runtime': 3362.4052, 'train_samples_per_second': 6.778, 'train_steps_per_second': 0.848, 'total_flos': 5954845409280000.0, 'train_loss': 2.519820521839878, 'epoch': 2.0})

After training is done you can save the model by calling `save_model()`. This will save the trained model to our `output_dir` from our `TrainingArguments`.

In [ ]:
trainer.save_model()

Saving model checkpoint to ./gpt2-reporter
Configuration saved in ./gpt2-reporter/config.json
Model weights saved in ./gpt2-reporter/pytorch_model.bin
Saving model checkpoint to ./gpt2-reporter
Configuration saved in ./gpt2-reporter/config.json
Model weights saved in ./gpt2-reporter/pytorch_model.bin
Several commits (3) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.30k/401M [00:00<?, ?B/s]

Upload file runs/Dec14_06-25-22_139d60cc267c/events.out.tfevents.1670999127.139d60cc267c.75.2:  45%|####5     …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/theta/gpt2-reporter
   3c6b01c..f9b411a  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/theta/gpt2-reporter
   3c6b01c..f9b411a  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}
To https://huggingface.co/theta/gpt2-reporter
   f9b411a..a51201b  main -> main

   f9b411a..a51201b  main -> main



In [ ]:
trainer.push_to_hub()

Saving model checkpoint to ./gpt2-reporter
Configuration saved in ./gpt2-reporter/config.json
Model weights saved in ./gpt2-reporter/pytorch_model.bin
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}


# Test the model

To test the model we are going to use another [highlight of the transformers library](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipelines) called `pipeline`. [Pipelines](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipelines) are objects that offer a simple API dedicated to several tasks, among others also `text-generation`

In [ ]:
from transformers import pipeline, AutoModelWithLMHead
model = AutoModelWithLMHead.from_pretrained('theta/gpt2-reporter')
reporter = pipeline('text-generation',model=model, tokenizer=MODEL_NAME)



/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--theta--gpt2-reporter/snapshots/a51201b45b615f4df14dd26bd438e5ddea4eca72/config.json
Model config GPT2Config {
  "_name_or_path": "theta/gpt2-reporter",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "output_past": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-gener

Downloading:   0%|          | 0.00/421M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--theta--gpt2-reporter/snapshots/a51201b45b615f4df14dd26bd438e5ddea4eca72/pytorch_model.bin
All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at theta/gpt2-reporter.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--uer--gpt2-chinese-cluecorpussmall/snapshots/7c87595de655dc7b0fbfaa545ae413a118063d0b/config.json
Model config GPT2Config {
  "_name_or_path": "uer/gpt2-chinese-cluecorpussmall",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "in

In [ ]:
reporter('總統宣布國防預算大漲BEG;END',max_length=800)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '總統宣布國防預算大漲BEG;END 美 國 就 航 母 建 設 提 出 要 求 。 雖 然 美 國 承 諾 暫 停 向 中 國 出 售 這 艘 航 空 母 艦 ， 但 到 目 前 為 止 ， 航 母 開 發 的 總 價 和 投 資 遠 遠 高 於 美 國 的 競 標 基 金 ， 中 國 主 導 的 航 母 項 目 ， 預 計 將 比 總 投 資 上 升 1000 % 左 右 是 否 能 夠 實 現 其 超 重 排 放 目 標 ？ 最 新 新 聞 報 道 稱 ， 由 於 美 國 還 想 繼 續 向 中 國 出 售 現 有 的 航 母 設 施 ， 因 此 新 增 預 算 可 能 超 過 美 國 其 他 預 算 ， 其 中 包 括 對 航 空 母 艦 的 投 資 。 這 其 中 包 括 建 造 更 多 的 新 型 號 航 母 、 改 進 和 更 強 大 的 艦 載 機 系 統 以 及 對 抗 性 巡 航 導 彈 等 。 美 國 主 管 國 防 預 算 的 副 國 務 卿 蓬 佩 奧 表 示 ， 即 使 美 國 無 法 在 本 屆 歐 洲 戰 略 盟 友 日 的 行 動 中 參 與 軍 事 行 動 也 能 夠 解 決 美 國 的 航 母 動 亂 的 問 題 ， 對 於 中 國 來 說 也 極 為 關 鍵 ， 美 國 將 會 面 對 更 多 的 挑 戰 。 外 界 普 遍 預 測 ， 除 非 航 母 出 現 爆 炸 危 機 ， 中 國 可 能 會 大 力 度 投 資 航 母 。 因 此 ， 蓬 佩 奧 提 到 了 大 約 60 萬 美 元 ， 來 向 美 國 「 要 回 金 錢 」 。 蓬 佩 奧 說 ， 由 於 航 母 項 目 的 支 出 將 遠 遠 超 過 美 國 預 算 ， 中 國 自 己 想 發 起 的 軍 事 行 動 能 夠 實 現 其 超 重 排 放 目 標 ， 對 於 美 國 也 極 為 關 鍵 。 蓬 佩 奧 說 ， 外 國 主 管 國 防 預 算 的 副 國 務 卿 蓬 佩 奧 在 周 五 （ 10 月 27 日 ） 上 午 就 軍 演 的 具 體 行 動 承 諾 給 中 國 提 供 了 建 議 ， 美 國 需 要 在 今 後 多 長 時 間 上 對 航 母 動 亂 提 出 更 多 的 抗 議 。 他 還 說 ， 如

In [ ]:
reporter('總統宣布國防預算大漲',max_length=800)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '總統宣布國防預算大漲 30 億 美 元 ， 美 國 政 府 的 貨 幣 刺 激 政 策 正 進 入 蜜 月 期 。 因 此 ， 儘 管 美 國 對 中 國 實 施 「 財 政 緊 縮 」 以 應 對 經 濟 增 長 放 緩 ， 中 國 是 否 仍 然 採 取 一 定 措 施 、 遏 制 以 中 國 國 利 貸 為 基 礎 實 施 的 貿 易 戰 仍 在 進 行 中 。 貿 易 戰 對 經 濟 的 影 響 ？ 由 於 全 球 經 濟 陷 入 衰 退 ， 中 國 在 2017 年 實 際 對 外 開 放 進 口 總 值 為 328 億 美 元 ， 比 去 年 增 長 了 0. 8 % ， 比 2007 全 年 增 長 2. 9 % ， 但 是 到 了 2019 年 ， 中 國 對 外 開 放 仍 然 以 增 長 3. 5 % 的 速 度 邁 上 3 % 的 台 階 。 一 些 評 論 人 士 認 為 ， 中 國 的 經 濟 增 長 也 進 入 了 「 瓶 頸 」 。 因 為 全 球 經 濟 的 衰 退 ， 中 國 在 去 年 的 增 速 在 7. 6 % 以 下 ， 而 且 今 年 的 經 濟 增 速 也 下 滑 了 3 % 。 中 國 政 府 宣 佈 的 「 財 政 緊 縮 」 政 策 是 根 據 中 國 國 家 統 計 局 公 布 的 經 濟 增 長 指 標 結 果 進 行 計 算 的 。 其 中 主 要 分 析 指 出 ， 中 國 經 濟 的 確 很 差 ， 在 這 種 長 期 惡 化 的 狀 況 下 ， 如 果 要 使 經 濟 繼 續 保 持 「 快 速 復 蘇 」 ， 就 必 須 依 靠 大 量 財 政 來 支 撐 。 中 國 財 政 緊 縮 是 如 何 造 成 的 ？ 財 政 部 周 三 就 《 中 國 共 產 黨 新 聞 工 作 條 例 》 開 展 聯 合 調 查 ， 並 在 官 方 微 博 @ 國 防 觀 察 網 刊 登 新 聞 稿 表 示 ， 雖 然 官 方 的 經 濟 調 查 顯 示 ， 中 國 經 濟 在 國 際 經 濟 復 蘇 衰 退 的 狀 況 下 能 在 2017 年 年 底 達 到 4 % ， 但 是 經 濟 卻 出 現 了 不 同 程 度 回 落 。 分 析 人 士 指 出 ， 中 國 經 濟 正